In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
import os
from tqdm import tqdm 
from sklearn.preprocessing import LabelEncoder

In [2]:
# Set device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [3]:
# Custom dataset class with label encoding
class VehicleDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        
        # Initialize label encoder
        self.label_encoder = LabelEncoder()
        self.dataframe['Label'] = self.label_encoder.fit_transform(self.dataframe['Label'])

        # Store mapping of encoded labels to original labels
        self.idx_to_label = {idx: label for idx, label in enumerate(self.label_encoder.classes_)}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['Filepath']
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return self.__getitem__((idx + 1) % len(self.dataframe))
        
        label = self.dataframe.iloc[idx]['Label']
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

    def get_class_names(self):
        return list(self.idx_to_label.values())

In [4]:
# Ensure directories exist
os.makedirs('E:/Vehicle/Cuda/models', exist_ok=True)
os.makedirs('E:/Vehicle/Cuda/results', exist_ok=True)

In [5]:
# Load data
train_df = pd.read_csv('E:/Vehicle/train_df.csv')
test_df = pd.read_csv('E:/Vehicle/test_df.csv')

In [6]:

# Define transformations
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
# Create datasets and dataloaders
train_dataset = VehicleDataset(train_df, transform=transform)
test_dataset = VehicleDataset(test_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
# Load pre-trained InceptionV3 model
model = models.inception_v3(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Linear(128, 5),  # Adjust output classes based on your dataset
    nn.LogSoftmax(dim=1)
)
model = model.to(device)

C:\Users\anand\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\anand\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [10]:
def train_model(model, criterion, optimizer, train_loader, num_epochs=30):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        tqdm_loader = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
        for images, labels in tqdm_loader:
            images, labels = images.to(device), torch.tensor(labels).to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            outputs, _ = model(images)  # Extract logits from InceptionOutputs
            
            # Calculate loss
            loss = criterion(outputs, labels.long())  # Ensure labels are cast to torch.long
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Update statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
            tqdm_loader.set_postfix(loss=running_loss / len(tqdm_loader), accuracy=100 * correct / total)
        
        # Print epoch statistics
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%')

# Example usage
train_model(model, criterion, optimizer, train_loader)


Epoch 1/30:   0%|                                                                               | 0/19 [00:00<?, ?it/s]C:\Users\anand\AppData\Local\Temp\ipykernel_1228\4153834551.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images, labels = images.to(device), torch.tensor(labels).to(device)
Epoch 1/30:  26%|███████████▌                                | 5/19 [01:19<03:23, 14.52s/it, accuracy=28.1, loss=0.412]

Error loading image E:\Vehicle\Vehicle_5_classes_sample\Bus\20210518_14_58_52_000_RMkLr8rWJDMBfnDv6WRtqfoMLT83_F_3264_2448.jpg: image file is truncated (111 bytes not processed)


Epoch 1/30:  37%|████████████████▏                           | 7/19 [01:45<02:45, 13.77s/it, accuracy=31.7, loss=0.568]

Error loading image E:\Vehicle\Vehicle_5_classes_sample\Truck\Datacluster Truck (123).jpg: image file is truncated (8 bytes not processed)


Epoch 1/30:  47%|████████████████████▊                       | 9/19 [02:11<02:13, 13.39s/it, accuracy=33.7, loss=0.717]

...

Epoch 30/30:  58%|███████████████████████▋                 | 11/19 [02:17<01:39, 12.50s/it, accuracy=98.6, loss=0.0272]

Error loading image E:\Vehicle\Vehicle_5_classes_sample\Tempo Traveller\20210517_06_39_54_000_bsfUf1M1E9bMsvY4ENiWwUWjDQp2_F_4160_3120.jpg: image file is truncated (15 bytes not processed)


Epoch 30/30:  74%|██████████████████████████████▏          | 14/19 [02:54<01:01, 12.39s/it, accuracy=98.4, loss=0.0385]

Error loading image E:\Vehicle\Vehicle_5_classes_sample\Auto\Datacluster Auto (61).jpg: image file is truncated (43 bytes not processed)


Epoch [30/30], Loss: 0.0816, Accuracy: 98.15%


In [11]:
# Save model
torch.save(model.state_dict(), 'E:/Vehicle/model/inceptionv3_cuda_model.pth')

In [13]:
# Evaluation
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), torch.tensor(labels).to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(f'Test Accuracy: {100 * correct / total}%')

C:\Users\anand\AppData\Local\Temp\ipykernel_1228\3547054473.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images, labels = images.to(device), torch.tensor(labels).to(device)


Test Accuracy: 59.73154362416108%
